# Data Input

To do any computation, you need to have data. Getting the data in the framework of a workflow is therefore the first step of every analysis. Nipype provides many different modules to grab or select the data:

    DataFinder
    DataGrabber
    FreeSurferSource
    JSONFileGrabber
    S3DataGrabber
    SSHDataGrabber
    SelectFiles
    XNATSource

This tutorial will only cover some of them. For the rest, see the section [``interfaces.io``](http://nipype.readthedocs.io/en/latest/interfaces/generated/nipype.interfaces.io.html) on the official homepage.

# Dataset structure

To be able to import data, you first need to be aware about the structure of your dataset. The structure of the dataset for this tutorial is according to BIDS, and looks as follows:

	TODO
    ds102
    ├── CHANGES
    ├── dataset_description.json
    ├── participants.tsv
    ├── README
    ├── sub-01
    │   ├── anat
    │   │   └── sub-01_T1w.nii.gz
    │   └── func
    │       ├── sub-01_task-flanker_run-1_bold.nii.gz
    │       ├── sub-01_task-flanker_run-1_events.tsv
    │       ├── sub-01_task-flanker_run-2_bold.nii.gz
    │       └── sub-01_task-flanker_run-2_events.tsv
    ├── sub-02
    │   ├── anat
    │   │   └── sub-02_T1w.nii.gz
    │   └── func
    │       ├── sub-02_task-flanker_run-1_bold.nii.gz
    │       ├── sub-02_task-flanker_run-1_events.tsv
    │       ├── sub-02_task-flanker_run-2_bold.nii.gz
    │       └── sub-02_task-flanker_run-2_events.tsv
    ├── sub-03
    │   ├── anat
    │   │   └── sub-03_T1w.nii.gz
    │   └── func
    │       ├── sub-03_task-flanker_run-1_bold.nii.gz
    │       ├── sub-03_task-flanker_run-1_events.tsv
    │       ├── sub-03_task-flanker_run-2_bold.nii.gz
    │       └── sub-03_task-flanker_run-2_events.tsv
    ├── ...
    .
    └── task-flanker_bold.json

# DataGrabber

``DataGrabber`` is a generic data grabber module that wraps around ``glob`` to select your neuroimaging data in an intelligent way. As an example, let's assume we want to grab the anatomical and functional images of a certain subject.

First, we need to create the ``DataGrabber`` node. This node needs to have some input fields for all dynamic parameters (e.g. subject identifier, task identifier), as well as the two desired output fields ``anat`` and ``func``.

In [1]:
from nipype import DataGrabber, Node

# Create DataGrabber node
dg = Node(DataGrabber(infields=['subject_id', 'task_name'],
                      outfields=['anat', 'func']),
          name='datagrabber')

# Location of the dataset folder
dg.inputs.base_directory = '/data/ds000114'

# Necessary default parameters
dg.inputs.template = '*'
dg.inputs.sort_filelist = True

Second, we know that the two files we desire are the the following location:

    anat = /data/ds000114/sub-01/anat/sub-01_T1w.nii.gz
    func = /data/ds000114/sub-01/func/sub-01_task-fingerfootlips_bold.nii.gz

We see that the two files only have two dynamic parameters between subjects and task names:

    subject_id: in this case 'sub-01'
    task_name: in this case fingerfootlips

This means that we can rewrite the paths as follows:

    anat = /data/ds102/[subject_id]/anat/[subject_id]_T1w.nii.gz
    func = /data/ds102/[subject_id]/func/[subject_id]_task-[task_name]_bold.nii.gz

Therefore, we need the parameter ``subject_id`` for the anatomical image and the parameter ``subject_id`` and ``task_name`` for the functional image. In the context of DataGabber, this is specified as follows:

In [2]:
dg.inputs.template_args = {'anat': [['subject_id']],
                           'func': [['subject_id', 'task_name']]}

Now, comes the most important part of DataGrabber. We need to specify the template structure to find the specific data. This can be done as follows.

In [3]:
dg.inputs.field_template = {'anat': 'sub-%02d/anat/*_T1w.nii.gz',
                            'func': 'sub-%02d/func/*task-%s_bold.nii.gz'}

TODO
You'll notice that we use ``%s``, ``%02d`` and ``*`` for placeholders in the data paths. ``%s`` is a placeholder for a string and is filled out by ``subject_id``. ``%02d`` is a placeholder for a integer number and is filled out by ``task_id``. ``*`` is used as a wild card, e.g. a placeholder for any possible string combination. This is all to set up the ``DataGrabber`` node.

TODO Now it is up to you how you want to feed the dynamic parameters into the node. You can either do this by using another node (e.g. ``IdentityInterface``) and feed ``subject_id`` and ``task_id`` as connections to the ``DataGrabber`` node or specify them directly as node inputs.

In [4]:
# Using the IdentityInterface
from nipype import IdentityInterface
infosource = Node(IdentityInterface(fields=['subject_id', 'task_name']),
                  name="infosource")
infosource.inputs.task_name = "fingerfootlips"
subject_id_list = [1, 2]
infosource.iterables = [('subject_id', subject_id_list)]

Now you only have to connect ``infosource`` with your ``DataGrabber`` and run the workflow to iterate over subjects 1, 2 and 3.

If you specify the inputs to the ``DataGrabber`` node directly, you can do this as follows:

In [5]:
# Specifying the input fields of DataGrabber directly
dg.inputs.subject_id = 1
dg.inputs.task_name = "fingerfootlips"

Now let's run the ``DataGrabber`` node and let's look at the output:

In [6]:
dg.run().outputs

170716-02:05:13,598 workflow INFO:
	 Executing node datagrabber in dir: /tmp/tmpphaq17mj/datagrabber



anat = /data/ds000114/sub-01/anat/sub-01_T1w.nii.gz
func = /data/ds000114/sub-01/func/sub-01_task-fingerfootlips_bold.nii.gz

# SelectFiles

`SelectFiles` is a more flexible alternative to `DataGrabber`. It uses the {}-based string formating syntax to plug values into string templates and collect the data. These templates can also be combined with glob wild cards. The field names in the formatting template (i.e. the terms in braces) will become inputs fields on the interface, and the keys in the templates dictionary will form the output fields.

Let's focus again on the data we want to import:

    anat = /data/ds000114/sub-01/anat/sub-01_T1w.nii.gz
    func = /data/ds000114/sub-01/func/sub-01_task-fingerfootlips_bold.nii.gz
    
Now, we can replace those paths with the accoridng {}-based strings.

    anat = /data/ds000114/sub-{subject_id}/anat/sub-{subject_id}_T1w.nii.gz
    func = /data/ds000114/sub-{subject_id}/func/sub-{subject_id}_task-{task_name}_bold.nii.gz

How would this look like as a `SelectFiles` node?

In [10]:
from nipype import SelectFiles, Node

# String template with {}-based strings
templates = {'anat': 'sub-{subject_id}/anat/sub-{subject_id}_T1w.nii.gz',
             'func': 'sub-{subject_id}/func/sub-{subject_id}_task-{task_name}_bold.nii.gz'}

# Create SelectFiles node
sf = Node(SelectFiles(templates),
          name='selectfiles')

# Location of the dataset folder
sf.inputs.base_directory = '/data/ds000114'

# Feed {}-based placeholder strings with values
sf.inputs.subject_id = '01'
sf.inputs.task_name = 'fingerfootlips'

Let's check if we get what we wanted.

In [11]:
sf.run().outputs

170716-02:13:43,303 workflow INFO:
	 Executing node selectfiles in dir: /tmp/tmpzr_3jkhd/selectfiles



anat = /data/ds000114/sub-01/anat/sub-01_T1w.nii.gz
func = /data/ds000114/sub-01/func/sub-01_task-fingerfootlips_bold.nii.gz

Perfect! But why is `SelectFiles` more flexible than `DataGrabber`? First, you perhaps noticed that with the {}-based string, we can reuse the same input (e.g. `subject_id`) multiple time in the same string, without feeding it multiple times into the template.

Additionally, you can also select multiple files without the need of an iterable node. For example, let's assume we want to select both anatomical images (`'sub-01'` and `'sub-02'`) at once. We can do this by using the following file template:

    'sub-0[1,2]/anat/sub-0[1,2]_T1w.nii.gz'

Let's see how this works:

In [28]:
from nipype import SelectFiles, Node
from os.path import abspath as opap

# String template with {}-based strings
templates = {'anat': 'sub-0[1,2]/anat/sub-0[1,2]_T1w.nii.gz'}


# Create SelectFiles node
sf = Node(SelectFiles(templates),
          name='selectfiles')

# Location of the dataset folder
sf.inputs.base_directory = '/data/ds000114'

# Print SelectFiles output
sf.run().outputs

170716-02:23:29,661 workflow INFO:
	 Executing node selectfiles in dir: /tmp/tmpv6q8umoi/selectfiles



anat = ['/data/ds000114/sub-01/anat/sub-01_T1w.nii.gz', '/data/ds000114/sub-02/anat/sub-02_T1w.nii.gz']

As you can see, now `anat` contains two file paths, one for the first and one for the second subject. As a side node, you could have also gotten them same thing with the wild card `*`:

    'sub-0*/anat/sub-0*_T1w.nii.gz'

## FreeSurferSource

***Note: FreeSurfer and the recon-all output is not included in this tutorial.***

`FreeSurferSource` is a specific case of a file grabber that felicitates the data import of outputs from the FreeSurfer recon-all algorithm. This of course requires that you've already run `recon-all` on your subject.

Before you can run `FreeSurferSource`, you first have to specify the path to the FreeSurfer output folder, i.e. you have to specify the SUBJECTS_DIR variable. This can be done as follows:

In [29]:
from nipype.interfaces.freesurfer import FSCommand
from os.path import abspath as opap

# Path to your freesurfer output folder
fs_dir = opap('/data/ds000114/derivatives/freesurfer')

# Set SUBJECTS_DIR
FSCommand.set_default_subjects_dir(fs_dir)

To create the `FreeSurferSource` node, do as follows:

In [30]:
from nipype import Node
from nipype.interfaces.io import FreeSurferSource

# Create FreeSurferSource node
fssource = Node(FreeSurferSource(subjects_dir=fs_dir),
                name='fssource')

Let's now run it for a specific subject.

In [34]:
fssource.inputs.subject_id = 'sub-01'
result = fssource.run() #TODO: doesn't work! I also cant run datalad get files that are reported to not exist

170716-02:30:13,899 workflow INFO:
	 Executing node fssource in dir: /tmp/tmpicgq9wsr/fssource
170716-02:30:13,909 root ERROR:
	 Internal Python error in the inspect module.
Below is the traceback from this internal error.

Traceback (most recent call last):
  File "/opt/conda/lib/python3.5/site-packages/IPython/core/interactiveshell.py", line 2881, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-34-6491b09a7842>", line 2, in <module>
    result = fssource.run() #TODO: doesn't work! I also cant run datalad get files that are reported to not exist
  File "/opt/conda/lib/python3.5/site-packages/nipype/pipeline/engine/nodes.py", line 372, in run
    self._run_interface()
  File "/opt/conda/lib/python3.5/site-packages/nipype/pipeline/engine/nodes.py", line 480, in _run_interface
    old_cwd = os.getcwd()
FileNotFoundError: [Errno 2] No such file or directory

During handling of the above exception, another exception occurred:

Traceback (most recent

FileNotFoundError: [Errno 2] No such file or directory

Did it work? Let's try to access multiple FreeSurfer outputs:

In [32]:
print('aparc_aseg: %s\n' % result.outputs.aparc_aseg)
print('brainmask: %s\n' % result.outputs.brainmask)
print('inflated: %s\n' % result.outputs.inflated)

aparc_aseg: <undefined>

brainmask: <undefined>

inflated: <undefined>



It seems to be working as it should. But as you can see, the `inflated` output actually contains the file location for both hemispheres. With `FreeSurferSource` we can also restrict the file selection to a single hemisphere. To do this, we use the `hemi` input filed:

In [ ]:
fssource.inputs.hemi = 'lh'
result = fssource.run()

170302-17:50:13,835 workflow INFO:
	 Executing node fssource in dir: /tmp/tmpI0UTIX/fssource


Let's take a look again at the `inflated` output.

In [ ]:
result.outputs.inflated

u'/data/ds102/freesurfer/sub001/surf/lh.inflated'

Perfect!